---
# **3. GARCH(1,1) моделирование волатильности**

Волатильность кластеризуется — после больших движений цен она растёт.
GARCH (Generalized Autoregressive Conditional Heteroskedasticity) моделирует условную волатильность.

### Спецификация GARCH(1,1)

$$r_t = \mu + \epsilon_t, \quad \epsilon_t \sim N(0, h_t)$$

$$h_t = \omega + \alpha \epsilon_{t-1}^2 + \beta h_{t-1}$$

где:
- $h_t$ — условная дисперсия
- $\omega$ — константа (baseline volatility)
- $\alpha$ — ARCH коэффициент (влияние шока)
- $\beta$ — GARCH коэффициент (инерция волатильности)

При стационарности ($\alpha + \beta < 1$):
$$\sigma_\infty = \sqrt{\frac{\omega}{1 - \alpha - \beta}}$$

### Устойчивость

$$\text{persistence} = \alpha + \beta$$

- Устойчивость < 0.995 → волатильность mean-reverting (стационарна)
- Устойчивость ≈ 1 → IGARCH (интегрированная GARCH)
- Устойчивость > 1 → нестационарна (модель неадекватна)

### Критерии корректности

1. $\omega > 0$
2. $\alpha, \beta \geq 0$
3. $\alpha > 10^{-8}$ (ARCH компонента существенна)
4. Устойчивость < 0.995 (стационарность)
5. Оптимизация сошлась (convergence_flag = 0)

Если какой-то критерий не выполнен → используется Sample волатильность (fallback).

### Информационные критерии

$$\text{AIC} = -2 \ln L + 2k, \quad \text{BIC} = -2 \ln L + k \ln n$$

(меньше → лучше модель)

### Нормализация

$$\sigma_\infty^{\text{annual}} = \sigma_\infty \times \sqrt{252} \times 100\%$$

---

In [ ]:
ANNUALIZATION_FACTOR = np.sqrt(252)
MIN_OBS = 100
PERSISTENCE_THRESHOLD = 0.995
ALPHA_MIN_THRESHOLD = 1e-8

print(f" Параметры: {returns.shape[1]} активов, {returns.shape[0]} наблюдений\n")

garch_params = {}
long_run_vols = {}
conditional_vols = {}
failed_tickers = []


for ticker in returns.columns:
    try:
        r = returns[ticker].dropna()


        if len(r) < MIN_OBS:
            failed_tickers.append(ticker)
            long_run_vols[ticker] = np.nan
            conditional_vols[ticker] = np.nan
            print(f"  {ticker}: недостаточно данных ({len(r)} < {MIN_OBS})")
            continue

        model = arch_model(r, vol='Garch', p=1, q=1, rescale=False)
        fitted = model.fit(disp='off', show_warning=False)


        if fitted.convergence_flag != 0:
            failed_tickers.append(ticker)
            long_run_vols[ticker] = np.nan
            conditional_vols[ticker] = np.nan
            print(f" {ticker}: оптимизация не сошлась (flag={fitted.convergence_flag})")
            continue

        omega = fitted.params['omega']
        alpha = fitted.params['alpha[1]']
        beta = fitted.params['beta[1]']
        persistence = alpha + beta

        if omega <= 0 or alpha < -1e-10 or beta < -1e-10:
            failed_tickers.append(ticker)
            long_run_vols[ticker] = np.nan
            conditional_vols[ticker] = np.nan
            print(f" {ticker}: отрицательные/некорректные параметры (ω={omega:.2e}, α={alpha:.2e}, β={beta:.2e})")
            continue

        if alpha < ALPHA_MIN_THRESHOLD:
            failed_tickers.append(ticker)
            long_run_vols[ticker] = np.nan
            conditional_vols[ticker] = np.nan
            print(f"  {ticker}: α практически нулевой ({alpha:.2e}), модель неадекватна")
            continue


        is_stationary = (persistence < PERSISTENCE_THRESHOLD)


        if is_stationary:
            long_run_vol = np.sqrt(omega / (1 - persistence)) * ANNUALIZATION_FACTOR * 100

            cond_vol_mean = fitted.conditional_volatility.mean() * ANNUALIZATION_FACTOR * 100

            status = "✅"
        else:
            long_run_vol = np.nan
            cond_vol_mean = np.nan
            status = "❌"
            failed_tickers.append(ticker)


        garch_params[ticker] = {
            'omega': omega, 'alpha': alpha, 'beta': beta, 'persistence': persistence
        }
        long_run_vols[ticker] = long_run_vol
        conditional_vols[ticker] = cond_vol_mean


        sample_vol = annual_vol[ticker] * 100

        aic = fitted.aic
        bic = fitted.bic

        if sample_vol > 0.01:
            ratio = long_run_vol / sample_vol
            ratio_str = f"({ratio:.2f}x)"
        else:
            ratio_str = "(N/A)"


        pers_str = f"α+β={persistence:.4f}"
        vol_str = f"{long_run_vol:.2f}%"
        comp_str = f"Sample={sample_vol:.2f}% {ratio_str}"

        print(f"{status} {ticker}: {pers_str} → σ_∞={vol_str} | {comp_str}")
        print(f"           AIC={aic:.1f}, BIC={bic:.1f} | α={alpha:.2e}, β={beta:.4f}")


    except Exception as e:
        failed_tickers.append(ticker)
        long_run_vols[ticker] = np.nan
        conditional_vols[ticker] = np.nan
        error_msg = str(e)[:60] if str(e) else "Unknown error"
        print(f" {ticker}: {error_msg}")

long_run_vols_series = pd.Series(long_run_vols)
conditional_vols_series = pd.Series(conditional_vols)
valid_vols = long_run_vols_series.dropna()

n_total = len(returns.columns)
n_valid = len(valid_vols)


print(f" Статистика:")
print(f"   Всего активов: {n_total}")
print(f"   Успешно откалибровано GARCH: {n_valid} ({n_valid/n_total*100:.0f}%)")
print(f"   Проблемных/неадекватных: {len(failed_tickers)} ({len(failed_tickers)/n_total*100:.0f}%)")


if len(valid_vols) > 0:
    print(f"\n Долгосрочные волатильности (σ_∞) из GARCH:\n")
    for ticker in valid_vols.sort_values(ascending=False).index:
        sample_vol = annual_vol[ticker] * 100
        garch_vol = valid_vols[ticker]
        ratio = garch_vol / sample_vol if sample_vol > 1.0 else 0
        print(f"   {ticker:6s}: σ_∞={garch_vol:>6.2f}% | Sample={sample_vol:>6.2f}% | Ratio={ratio:>5.2f}x")

print(f"\n Диагностика проблемных активов:\n")


if len(garch_params) > 0:
    params_df = pd.DataFrame(garch_params).T

    for ticker in failed_tickers:
        if ticker in params_df.index:
            pers = params_df.loc[ticker, 'persistence']
            omega = params_df.loc[ticker, 'omega']
            alpha = params_df.loc[ticker, 'alpha']
            beta = params_df.loc[ticker, 'beta']

            if pers >= PERSISTENCE_THRESHOLD:
                print(f"   {ticker}: нестационарна (α+β={pers:.4f} ≥ {PERSISTENCE_THRESHOLD})")
            elif alpha < ALPHA_MIN_THRESHOLD:
                print(f"   {ticker}: α практически нулевой ({alpha:.2e}) → ARCH компонента отсутствует")
            elif not (omega > 0):
                print(f"   {ticker}: ω ≤ 0 ({omega:.2e})")
            else:
                print(f"   {ticker}: другая причина")
        else:
            print(f"   {ticker}: недостаточно данных или ошибка оптимизации")
else:
    print("     Нет успешно откалиброванных активов")


volatilities_final = {}
garch_usage = {}

for ticker in returns.columns:
    if not np.isnan(long_run_vols[ticker]):
        volatilities_final[ticker] = long_run_vols[ticker]
        garch_usage[ticker] = "GARCH"
    else:
        volatilities_final[ticker] = annual_vol[ticker] * 100
        garch_usage[ticker] = "Sample"


volatilities_final_series = pd.Series(volatilities_final)


print(f"\n Финальные волатильности:\n")
for ticker in volatilities_final_series.sort_values(ascending=False).index:
    source = garch_usage[ticker]
    symbol = "📊" if source == "GARCH" else "📈"
    print(f"   {symbol} {ticker:6s}: {volatilities_final_series[ticker]:>6.2f}% [{source}]")


print(f"\n   Используется GARCH: {sum(1 for x in garch_usage.values() if x == 'GARCH')}/{n_total}")
print(f"   Используется Sample (fallback): {sum(1 for x in garch_usage.values() if x == 'Sample')}/{n_total}")

garch_results = {
    'params': pd.DataFrame(garch_params).T if len(garch_params) > 0 else pd.DataFrame(),
    'long_run_vols': long_run_vols_series,
    'conditional_vols': conditional_vols_series,
    'volatilities_final': volatilities_final_series,
    'garch_usage': garch_usage,
    'failed_tickers': failed_tickers
}


print(f" Переменные:")
print(f"   garch_results['params']           : параметры GARCH (ω, α, β, persistence)")
print(f"   garch_results['long_run_vols']    : долгосрочные волатильности σ_∞")
print(f"   garch_results['conditional_vols'] : средние условные волатильности")
print(f"   garch_results['volatilities_final'] : финальные волатильности (GARCH + Sample)")
print(f"   garch_results['garch_usage']      : источник каждой волатильности")
print(f"   garch_results['failed_tickers']   : список неудачных активов\n")
print(f" ИТОГО:")
print(f"    Успешно откалибровано: {n_valid}/{n_total}")
print(f"     Используется Sample fallback: {n_total - n_valid}/{n_total}")